## Data Cleaning

Now since we have extracted data from the website, it is not cleaned and ready to be analyzed yet. The reviews section will need to be cleaned for punctuations, spellings and other characters. 

In [50]:
#imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

#regex
import re

In [4]:
#create a dataframe from csv file

cwd = os.getcwd()

df = pd.read_csv(cwd+"/BA_reviews.csv", index_col=0)

In [6]:
df.head()

,reviews,stars,date,country
0,✅ Trip Verified | My family and I were booke...,5.0,9th August 2024,United Kingdom
1,Not Verified | We had to change from AA to BA...,1.0,8th August 2024,United States
2,✅ Trip Verified | After paying $6500 for ticke...,2.0,8th August 2024,United States
3,✅ Trip Verified | An excellent flight on BA Ci...,2.0,7th August 2024,Lebanon
4,✅ Trip Verified | Crew were amazing and atte...,8.0,7th August 2024,United Kingdom


We will also create a column which mentions if the user is verified or not. 

In [9]:
df['verified'] = df.reviews.str.contains("Trip Verified")

In [11]:
df['verified']

0        True
1       False
2        True
3        True
4        True
        ...  
3495    False
3496    False
3497    False
3498    False
3499    False
Name: verified, Length: 3500, dtype: bool

### Cleaning Reviews

We will extract the column of reviews into a separate dataframe and clean it for semantic analysis

In [15]:
#for lemmatization of words we will use nltk library
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()


reviews_data = df.reviews.str.strip("✅ Trip Verified |")

#create an empty list to collect cleaned data corpus
corpus =[]

#loop through each review, remove punctuations, small case it, join it and add it to corpus
for rev in reviews_data:
    rev = re.sub('[^a-zA-Z]',' ', rev)
    rev = rev.lower()
    rev = rev.split()
    rev = [lemma.lemmatize(word) for word in rev if word not in set(stopwords.words("english"))]
    rev = " ".join(rev)
    corpus.append(rev)

In [17]:
# add the corpus to the original dataframe

df['corpus'] = corpus

In [19]:
df.head()

,reviews,stars,date,country,verified,corpus
0,✅ Trip Verified | My family and I were booke...,5.0,9th August 2024,United Kingdom,True,family booked leave london hong kong august ma...
1,Not Verified | We had to change from AA to BA...,1.0,8th August 2024,United States,False,verified change aa ba flight london missed con...
2,✅ Trip Verified | After paying $6500 for ticke...,2.0,8th August 2024,United States,True,paying ticket family including pay sit togethe...
3,✅ Trip Verified | An excellent flight on BA Ci...,2.0,7th August 2024,Lebanon,True,excellent flight ba cityflyer plane clean comf...
4,✅ Trip Verified | Crew were amazing and atte...,8.0,7th August 2024,United Kingdom,True,crew amazing attentive nothing missed definite...


### Cleaning/Format date

In [23]:
df.dtypes

reviews      object
stars       float64
date         object
country      object
verified       bool
corpus       object
dtype: object

In [31]:
import pandas as pd

def custom_date_parser(date_str):
    formats = ["%Y-%m-%d", "%d/%m/%Y", "%B %d, %Y", "%d %B %Y", "%dth %B %Y", "%dst %B %Y", "%dnd %B %Y", "%drd %B %Y"]
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    raise ValueError(f"Invalid date format: {date_str}")

df['date'] = df['date'].apply(custom_date_parser)

    
df.date = pd.to_datetime(df.date)

In [33]:
df.date.head()

0   2024-08-09
1   2024-08-08
2   2024-08-08
3   2024-08-07
4   2024-08-07
Name: date, dtype: datetime64[ns]

### Cleaning ratings with stars

In [52]:
#check for unique values
df.stars.unique()

array([ 5.,  1.,  2.,  8., 10.,  3.,  6.,  9.,  7.,  4., nan])

In [54]:
df.stars.value_counts()

stars
1.0     861
2.0     406
3.0     403
8.0     339
10.0    284
7.0     275
9.0     273
5.0     245
4.0     236
6.0     175
Name: count, dtype: int64

There are 5 rows having values "None" in the ratings. We will drop all these 5 rows. 

In [61]:
# drop the rows where the value of ratings is None
df = df.replace(['nan', 'None', '', ' ', '\t', -1, 0, 999], np.nan)
df['stars'] = pd.to_numeric(df['stars'], errors='coerce')
df.dropna(subset=['stars'], inplace=True)

In [63]:
#check the unique values again
df.stars.unique()

array([ 5.,  1.,  2.,  8., 10.,  3.,  6.,  9.,  7.,  4.])

## Check for null Values

In [66]:
df.isnull().value_counts()

reviews  stars  date   country  verified  corpus
False    False  False  False    False     False     3495
                       True     False     False        2
Name: count, dtype: int64

In [72]:
df.dropna(subset=['country'], inplace=True)
df.country.isnull().value_counts()

country
False    3495
Name: count, dtype: int64

We have two missing values for country. For this we can just remove those two reviews (rows) from the dataframe. 

In [75]:
#drop the rows using index where the country value is null
df.drop(df[df.country.isnull() == True].index, axis=0, inplace=True)

In [77]:
df.shape

(3495, 6)

In [116]:
#resetting the index
df.reset_index(drop=True)

,reviews,stars,date,country,verified,corpus
0,Not Verified | Worst experience ever. Outbound...,5,2022-11-07,Italy,False,verified worst experience ever outbound flight...
1,✅ Trip Verified | Check in was a shambles at ...,1,2022-11-07,Malaysia,True,check shamble bwi counter open full flight bag...
2,✅ Trip Verified | Beyond disgusted with the fa...,5,2022-11-05,United Arab Emirates,True,beyond disgusted fact baggage yet delivered we...
3,✅ Trip Verified | On July 19th 2022 I had subm...,1,2022-10-31,United States,True,july th submitted complaint form regard fact b...
4,✅ Trip Verified | I booked the flight on Oct ...,1,2022-10-31,United States,True,booked flight oct cancel flight day learning g...
...,...,...,...,...,...,...
3406,This was a bmi Regional operated flight on a R...,1,2012-08-29,United Kingdom,False,bmi regional operated flight rj manchester hea...
3407,LHR to HAM. Purser addresses all club passenge...,10,2012-08-28,United Kingdom,False,lhr ham purser address club passenger name boa...
3408,My son who had worked for British Airways urge...,10,2011-10-12,United Kingdom,False,son worked british airway urged fly british ai...
3409,London City-New York JFK via Shannon on A318 b...,8,2011-10-11,United States,False,london city new york jfk via shannon really ni...


*****

Now our data is all cleaned and ready for data visualization and data analysis.

In [79]:
# export the cleaned data

df.to_csv(cwd + "/cleaned-BA-reviews.csv")